In [3]:
import pandas as pd
from scipy.spatial import distance as dist
from datetime import timedelta

In [4]:
def getTimeStamp(timeStamp):
    return pd.to_datetime(timeStamp)

def createDF(folderName):
    flow_df = pd.read_csv(folderName + 'flow.tsv', delimiter='\t', header=None)
    speed_df = pd.read_csv(folderName + 'speed.tsv', delimiter='\t', header=None)
    occupancy_df = pd.read_csv(folderName + 'occupancy.tsv', delimiter='\t', header=None)
    
    df = pd.DataFrame({'A' : []})
    del df['A']
    for col in flow_df:
        df[col] = pd.Series(data = list(zip(flow_df[col],speed_df[col], occupancy_df[col])))
    
    timeStamp_df = pd.read_csv(folderName + 'timestamp.tsv', delimiter='\t', header=None)
    df['timeStamp'] = getTimeStamp(timeStamp_df[0])
    return df

In [5]:
def containsNeg(vec):
    if(vec[0] < 0):
        return 1
    elif(vec[1] < 0):
        return 1
    elif(vec[2] < 0):
        return 1
    else:
        return 0

In [6]:
def getVecInTimePeriod(timeInterval, currTime, df):
    timeInterval = timedelta(minutes = timeInterval)
    startTime = currTime - timeInterval
    endTime = currTime + timeInterval
    df = df[df['timeStamp'] >= startTime]
    return df[df['timeStamp'] <= endTime]

In [7]:
zone1160 = createDF('cleaning1/1160/')

In [12]:
result = getVecInTimePeriod(5, zone1160['timeStamp'][0], zone1160)
result

,0,1,2,timeStamp
0,"(8, 68.0, 3.0)","(13, 67.0, 4.0)","(255, 0.0, 4.0)",2006-09-01 00:00:07
1,"(9, 67.0, 4.0)","(4, 60.0, 2.0)","(255, 0.0, 4.0)",2006-09-01 00:01:07
2,"(11, 66.0, 4.0)","(10, 64.0, 5.0)","(255, 0.0, 4.0)",2006-09-01 00:02:07
3,"(-8, 71.0, 3.0)","(3, 63.0, 1.0)","(255, 0.0, 4.0)",2006-09-01 00:03:07
4,"(5, 65.0, 2.0)","(9, 70.0, 3.0)","(255, 0.0, 4.0)",2006-09-01 00:04:07


In [15]:
ex = zone1160[0][0]
#same lane calculations
def compareLaneData(currVec, currTime, df, distThreshold):
    row_index = 0
    col_index = 0
    numVecGreater = 0
    while row_index < df.shape[0]:
        edist = dist.euclidean(df[0][row_index], currVec)
        if(edist > distThreshold):
            numVecGreater += 1
        row_index += 1
    totalVec = row_index - 1
    return numVecGreater/totalVec

In [17]:
compareLaneData(zone1160[0][0], zone1160['timeStamp'][0], result, 5)

0.25